In [ ]:
import os
import sys
import time
import random
import numpy as np
from IPython.display import HTML, display

# Montage du projet dans Colab et installation des modules requis
from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/MyDrive/ai-park")
sys.path.insert(0, os.getcwd())
!pip install -r requirements.txt &> /dev/null

# imports du projets
from park.logic.grid import Grid2D
from park.logic.node import Node
from park.internal.math import Vector2D
from park.internal.collider import BoxCollider


# Qu'est-ce que la recherche de trajets ?

C'est la recherche d'un trajet qui permet d'aller d'un point A à un point B au travers d'une carte.

Les algorithmes que nous allons voir aujourd'hui sont les suivants.

1. Recherche profondeur d'abord (DFS)
2. Recherche Largeur d'abord (BFS)
3. Recherche par heuristique avec A* 

Pour avoir une bonne intuition sur ces algorithmes ouvre [ce lien](https://pathfindout.com/).

### Se déplacer en voiture au Québec

![Villes de la province de Québec](images/TSPquebec.png "Villes de la province de Québec")

Original de l'image disponible à https://freevectormaps.com/canada/quebec/CA-QC-EPS-01-0001?ref=atr


# Représantation d'une carte 2D en python

Il existe beaucoups de manières de représenter une carte en python. 

- Un tableau de valeurs en python. (numpy)
- Une liste de tuples
- Une double liste
- Un dictionnaire
- Etc.

Par exemple une carte de 10 en Y à la verticale et 10 en X à l'horizontal peut facilement être écrite en python comme suit:

In [ ]:
import numpy as np

# Ici, Y est en premier et représente les lignes et X est en deuxième représentant les colonnes
carte = np.zeros((11, 11))
print(carte)

In [ ]:
"""
Par la suite on peut manuellement ajouter des information pour chaque point en changeant des informations contenu dans le tableau. 

Par exemple, si on veut dire que le point de départ est à Y = 0 et X = 0 on peut assigner la valeur 1 comme suit:
"""
carte[0, 0] = 1
print(carte)

In [ ]:
"""
Si on veut ajouter un point d'arrivé à Y = 7, X = 9, on peut assigner la valeur 2 comme suit:
"""

carte[7, 9] = 2
print(carte)

In [ ]:
"""
Si on veut ajouter des obstacles dans la colone du centre de la carte, on peut assigner la valeur 3 comme suit:
"""
carte[:, 5] = 3
print(carte)

In [ ]:
"""
De même que si on veut ajouter des obstacles dans la ligne du centre, on peut assigner la valeur 3 comme suit:
"""
carte[5, :] = 3
print(carte)

In [ ]:
"""
Maintenant si on veut ajouter des obstacles aléatoires dans la carte, on peut assigner la valeur 4 comme suit:
Il se peut que l'obstacle aléatoire soit généré par dessus un autre dans la carte!
"""
random_Y = np.random.randint(0, 11)
random_X = np.random.randint(0, 11)
carte[random_Y, random_X] = "4"
print(carte)

In [ ]:
"""
Finalement, si on veut en ajouter 10 obstacles avec la valeur 5 ça donne :
"""
for i in range(10):
    random_Y = np.random.randint(0, 11)
    random_X = np.random.randint(0, 11)
    carte[random_Y, random_X] = 5
print(carte)

In [ ]:
"""
## ❗ RAPPEL : Peut-on ajouter ou lire une valeur à l'index 11, 11 ?
"""
carte[11, 11] = 6
print(carte[11, 11])

## Représentation avec `Grid2D`, `Node` et `Vector2D`

Pour sauver la Ronde 2.0 et ses robots, il faudrat se familiariser avec la représentation de la carte.
Dans l'ancien code de la Ronde 2.0 qu'il vous faudra réparer, trois structures de données sont très importantes pour les robots.


### 1. Les objets `Vector2D`

##### **Description:**
Le rôle des objets `Vector2D` est de représenter un point **x** et **y** dans la carte des robots.

##### **Attributs:**
- `x` : La coordonnée horizontale du point.
- `y` : La coordonnée verticale du point.

##### **Méthodes importantes:**
Plusieurs méthode sont déjà écrite qui vous permettent d'additionner, soustraire, multiplier et diviser deux points entre eux.
La méthode la plus importante pour les robots est celle qui permet le calcul de la distance entre deux points.

- **Distance entre deux points:**

    La méthode `distance_from(Vector2D)` permet de calculer la **distance entre deux points** dans la carte.
    Comme pour l'hypothénuse d'un triangle, la distance est calculée comme suit:
    
    ${distance} = \sqrt{(\Delta x)^2 + (\Delta y)^2}$

    qui est l'équivalent de :

    ${distance} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$

    La méthode s'applique à un point `Vector2D` et prend en paramètre un autre point `Vector2D`.

    Par exemple en code ça donne:  
    ```python
    point1 = Vector2D(0, 0)
    point2 = Vector2D(3, 4)
    print(point1.distance_from(point2))  # Résultat : 5.0
    ```

In [ ]:
# Affiche la distance entre un point en x = 23, y = 42 et un autre en x = 331, y = 71

# ====================== #
# Met ton code ici !


# ====================== #
print(f"Point1 = {point1}")
print(f"Point2 = {point2}")
print(f"Distance entre les deux = {point1.distance_from(point2)}")

# C'est trop facile ? affiche la distance entre deux points généré aléatoirement:
# ====================== #
# Met ton code ici !


# ====================== #
print(f"La distance entre les points {point1} et {point2} est {point1.distance_from(point2)}")

### 2. Les objets `Node`

##### **Description:**
Le rôle des objets `Node` est de représenter un case définie par sont point central **x** et **y** dans la carte des robots.
Cette zone peut être occupée ou libre, indiquant ainsi si elle contient d'autres robots, des manèges ou bien des visiteurs.

##### **Attributs importants:**
- `center` : Les coordonnées centrales de la case défini par un objet `Vector2D`.

    Par exemple en code ça donne:  
    ```python
    case1 = Node(Vector2D(1, 2), 1) # Crée une case ayant un centre à 1, 2 et une taille de 2
    print(case1.center.x, case1.center.y)  # Résultat : 1.0, 2.0
    print(int(case1.center.x), int(case1.center.y))  # Résultat : 1, 2
    ```

##### **Méthodes importantes:**
La méthode la plus importante dans pour les robots est celle qui évalue si on peut marcher dans cette zone.

- **Peut-on marcher dans cette case ? :**

    La méthode `walkable` permet de savoir si **l'on peut marcher dans cette case**. 

    Par exemple en code ça donne:  
    ```python
    case1 = Node(Vector2D(0, 0), 1) # Crée une case ayant un centre à 0, 0 et une taille de 2
    print(f"Peut-on marcher dans la case1 ? {case1.walkable}")  # Résultat : True
    case1.occupants.append(BoxCollider)
    print(f"Peut-on marcher dans la case1 ? {case1.walkable}")  # Résultat : False
    ```

### 3. Les objets `Grid2D`

##### **Description:**
Le rôle des objets `Grid2D` est de représenter la carte du parc d'attraction en 2 dimensions. Elle utilise une liste imbriquée de cases `Node`, autrement dit c'est une liste dans une autre liste. Il faudrat donc accèder aux éléments avec deux opérateurs d'accès `[y][x]` pour arriver au bon élément. Cet objet est assez complexe mais vous utiliserez principalement un seul attribut et une seule méthode.


##### **Attributs:**
- `nodes` : La liste de liste d'objets `Nodes`. Cette liste représente la carte de cases sur laquelle les robots se déplacent.

Par exemple en code ça donne:  
```Python
case_1_1 = grid.nodes[1][1]
```

##### **Méthodes:**
- **Trouver les cases voisines d'une case**

    La méthode `get_node_neighbors(Node, allow_diagonal = False)` permet d'avoir une liste de toutes les cases voisines à une case dans la carte. Soit avec ou sans les cases en diagonales.

    Par exemple en code ça donne:  
    ```python
    cases_voisines = grid.get_node_neighbors(case1, allow_diagonal = False)
    print([voisin.center for voisin in cases_voisines])
    ```

# Crée ton labyrinthe!
On pourrait utiliser des chiffres pour créer un labyrinthe mais c'est bien plus beau avec des émoticônes!

J'ai utilisé l'algorithme de génération aléatoire disponible [ici](https://inventwithpython.com/recursion/chapter11.html).

In [ ]:
# based on https://inventwithpython.com/recursion/chapter11.html
import time
import numpy as np
from IPython.display import HTML, display

# Défini la taille du labyrinthe minimum 3x3 et impair
TAILLE_Y = 21
TAILLE_X = 21

assert TAILLE_Y % 2 == 1 and TAILLE_Y >= 3
assert TAILLE_X % 2 == 1 and TAILLE_X >= 3

# Permet de regénérer le même labyrinthe
import random
# SEED = 2
# random.seed(SEED)

VIDE = '⬜'
MUR = '🟦'
DEBUT = '🟩'
FIN = '🟥'
PACMAN = '😼'
VISITEE = '🟨'
CHEMIN = '🔳'
NORD, SUD, EST, OUEST = 'n', 's', 'e', 'o'

# Pour générer une carte vide
labyrinthe = np.full((TAILLE_Y, TAILLE_X), VIDE)
labyrinthe[TAILLE_Y-1, :] = MUR
labyrinthe[:, TAILLE_X-1] = MUR
labyrinthe[0, :] = MUR
labyrinthe[:, 0] = MUR

# On affiche le labyrinthe.
chaine_labyrinthe = '<br>'.join(''.join(row) for row in labyrinthe)
display(HTML(f'''
    <div id="maze" style="font-family: monospace; white-space: pre; line-height: 1.2;">
    {chaine_labyrinthe}
    </div>
    <script>
    document.getElementById('maze').innerHTML = `{chaine_labyrinthe}`;
    </script>
    '''))

# Fonction qui met à jour le labyrinthe
def afficherLabyrinthe(labyrinthe, x, y, cases_visitees):
    for c in cases_visitees:
      labyrinthe[int(c.y), int(c.x)] = VISITEE

    if (x, y) != (0, 0):
      labyrinthe[y, x] = PACMAN

    chaine_labyrinthe = '<br>'.join(''.join(ligne) for ligne in labyrinthe)

    html = f'''
    <script>
    document.getElementById('maze').innerHTML = `{chaine_labyrinthe}`;
    </script>
    '''
    display(HTML(html))
    time.sleep(0.05)


# Affiche la case de début et de fin
labyrinthe[1, 1] = DEBUT
labyrinthe[-2, -2] = FIN
afficherLabyrinthe(labyrinthe.copy(), 0, 0, [])

# Crée la carte dans un objet Grid2D pour les algorithmes de recherche.
grid = Grid2D(TAILLE_X, TAILLE_Y, 1.0)
for i in range(TAILLE_Y):
  for j in range(TAILLE_X):
    if labyrinthe[i, j] == MUR:
      grid.nodes[i][j].occupants.append(BoxCollider)

In [ ]:
# Fonction qui génère le labyrinthe aléatoirement
def genererLabyrinthe(x, y):
  labyrinthe[y, x] = VIDE
  afficherLabyrinthe(labyrinthe.copy(), x, y, [])

  voisins_non_visites = []
  while True:
    if y > 1 and (x, y - 2) not in cases_visitees:
      voisins_non_visites.append(NORD)
    if y < TAILLE_Y - 2 and (x, y + 2) not in cases_visitees:
      voisins_non_visites.append(SUD)
    if x < TAILLE_X - 2 and (x + 2, y) not in cases_visitees:
      voisins_non_visites.append(EST)
    if x > 1 and (x - 2, y) not in cases_visitees:
      voisins_non_visites.append(OUEST)

    # Si on se retrouve dans un cul de sac on arrête
    if len(voisins_non_visites) == 0:
      break
    else:
      # Choix d'un voisin au hasard
      prochaine_case = random.choice(voisins_non_visites)
      if prochaine_case == NORD:
        labyrinthe[y - 1, x] = VIDE
        y -= 2
      elif prochaine_case == SUD:
        labyrinthe[y + 1, x] = VIDE
        y += 2
      elif prochaine_case == EST:
        labyrinthe[y, x + 1] = VIDE
        x += 2
      elif prochaine_case == OUEST:
        labyrinthe[y, x - 1] = VIDE
        x -= 2
      cases_visitees.append((x, y))
      voisins_non_visites = []
      genererLabyrinthe(x, y)
      
      

# On rempli un tableau numpy avec des murs partout
labyrinthe = np.full((TAILLE_Y, TAILLE_X), MUR)


# Pour générer un labyrinthe dans la carte.
cases_visitees = [(1, 1)]
genererLabyrinthe(1, 1)

# Affiche la case de début et de fin
labyrinthe[1, 1] = DEBUT
labyrinthe[-2, -2] = FIN
afficherLabyrinthe(labyrinthe.copy(), 0, 0, [])

# Crée la carte dans un objet Grid2D pour les algorithmes de recherche.
grid = Grid2D(TAILLE_X, TAILLE_Y, 1.0)
for i in range(TAILLE_Y):
  for j in range(TAILLE_X):
    if labyrinthe[i, j] == MUR:
      grid.nodes[i][j].occupants.append(BoxCollider)

# Recherche Profondeur d'abord (DFS)

L'algorithme de recherche par profondeur d'abord va prioriser aller jusqu'au fond d'une branche avant de faire demi tour lorsqu'il a le choix.

In [ ]:
# On affiche le labyrinthe.
chaine_labyrinthe = '<br>'.join(''.join(row) for row in labyrinthe)
display(HTML(f'''
    <div id="maze" style="font-family: monospace; white-space: pre; line-height: 1.2;">
    {chaine_labyrinthe}
    </div>
    <script>
    document.getElementById('maze').innerHTML = `{chaine_labyrinthe}`;
    </script>
    '''))


# Fonction pour afficher le chemin trouvé
def afficherChemin(labyrinthe, chemin):
  for node in chemin:
    x = int(node.center.x)
    y = int(node.center.y)
    labyrinthe[y, x] = CHEMIN
    afficherLabyrinthe(labyrinthe, 0, 0, [])

# Fonction qui trouve le chemin entre
# une case visitée et la case départ
def trouverChemin(cases_visitees, cases_precedentes, case_actuelle):
  chemin = [case_actuelle]
  # Tant qu'on n'est pas rendu à la case départ (None)
  index_precedente = cases_visitees.index(case_actuelle)
  case_precedente = cases_precedentes[index_precedente]
  while case_precedente != None:
    # On ajoute la case précédente à la liste qui forme le chemin
    chemin.append(case_precedente)
    # On trouve l'index de la case précédente dans la listes de cases visitées
    index_precedente = cases_visitees.index(case_precedente)
    # On trouve la nouvelle case précédente dans la liste de cases précédentes
    case_precedente = cases_precedentes[index_precedente]

  # On retourne la liste de cases qui forme le chemin
  return chemin


# Fonction de recherche par profondeur d'abord
def dfs(grid):
  
  # liste de tuples cases à visiter & case précédente
  cases_a_visiter = [(grid.nodes[1][1], None)]
  
  # liste des cases déjà explorées
  cases_visitees = [] 
  
  # liste des cases précédentes aux cases explorées
  cases_precedentes = []

  # Tant qu'on a des cases à visiter
  while cases_a_visiter:
    
    # Choisir et retirer une case à visiter
    
    # Ajoute la case courante aux cases déjà visitées
    
    # Ajoute la case précédente à la case courante aux cases précédentes
    

    # Sort les coordonées x, y de la case courante
    
    
    # Affiche la case courante dans le graph
    afficherLabyrinthe(labyrinthe.copy(), x, y, [c.center for c in cases_visitees])

    # Si on trouve la sortie on sort.
    # Imprime le nombre de cases visitées et la longueur du chemin

    # Liste toutes les cases voisines à la case courante sans diagonales
    
    
    # Pour chaque voisin
    
      # Si on peut marcher dans le voisin et que le voisin n'a pas déjà été visité
      
        # Si le voisin n'est pas déjà dans la liste de case à visiter non plus
        
          # On ajoute le voisin à la liste de cases à visiter avec son parent (la case actuelle)
          
      
  # Si on a pas trouver la sortie alors on retourne un chemin vide!
  return False, []


# Ordre (y, x) pour grid
status, chemin = dfs(grid)

# afficher le chemin trouvé
afficherChemin(labyrinthe.copy(), chemin)

# Recherche Largeur d'abord (BFS)

L'algorithme de recherche par largeur d'abord va prioriser la case la plus près pas encore non-explorée entre toutes les branches qu'il a accès.

In [ ]:
# On affiche le labyrinthe.
chaine_labyrinthe = '<br>'.join(''.join(row) for row in labyrinthe)
display(HTML(f'''
    <div id="maze" style="font-family: monospace; white-space: pre; line-height: 1.2;">
    {chaine_labyrinthe}
    </div>
    <script>
    document.getElementById('maze').innerHTML = `{chaine_labyrinthe}`;
    </script>
    '''))

# Fonction de recherche par largeur d'abord c'est la même chose à 1 ligne de différence.
def bfs(grid):
  # liste de tuples cases à visiter & case précédente
  cases_a_visiter = [(grid.nodes[1][1], None)]
  
  # liste des cases déjà explorées
  cases_visitees = [] 
  
  # liste des cases précédentes aux cases explorées
  cases_precedentes = []

  # Tant qu'on a des cases à visiter
  while cases_a_visiter:
    
    # Choisir et retirer une case à visiter
    
    # Ajoute la case courante aux cases déjà visitées

    # Ajoute la case précédente à la case courante aux cases précédentes
    

    # Sort les coordonées x, y de la case courante
    
    
    # Affiche la case courante dans le graph
    afficherLabyrinthe(labyrinthe.copy(), x, y, [c.center for c in cases_visitees])

    # Si on trouve la sortie on sort.
    # Imprime le nombre de cases visitées et la longueur du chemin
    

    # Liste toutes les cases voisines à la case courante sans diagonales
    
    
    # Pour chaque voisin

      # Si on peut marcher dans le voisin et que le voisin n'a pas déjà été visité
      
        # Si le voisin n'est pas déjà dans la liste de case à visiter non plus
        
          # On ajoute le voisin à la liste de cases à visiter avec son parent (la case actuelle)
          
      
  # Si on a pas trouver la sortie alors on retourne un chemin vide!
  return False, []
              
status, chemin = bfs(grid)
afficherChemin(labyrinthe.copy(), chemin)
        
        

# Recherche par heurisque avec l'algorithme A* (A star)

L'algorithme de recherche A* introduit un coût pour savoir quelle case il doit prioriser dans la liste des cases à visiter. 

In [ ]:
# On affiche le labyrinthe.
chaine_labyrinthe = '<br>'.join(''.join(row) for row in labyrinthe)
display(HTML(f'''
    <div id="maze" style="font-family: monospace; white-space: pre; line-height: 1.2;">
    {chaine_labyrinthe}
    </div>
    <script>
    document.getElementById('maze').innerHTML = `{chaine_labyrinthe}`;
    </script>
    '''))

# Fonction qui estime le coût entre la position finale et la position courante
def heuristique(position, fin):
  # Changer la fonction d'heuristique ici
  cost = 0
  return cost

# Fonction qui calcul le coût d'une case
def fonctionDeCout(cout_actuel, cout_estime):
  # La fonction de cout est le cout actuel + le cout estimé par heuristique
  return cout_actuel + cout_estime

def rechercheHeuristiqueAStar(grid, debut, fin):
  # Initialisation des listes
  cases_a_visiter = [(debut, None)]
  cases_visitees = []
  cases_precedentes = []

  # Tant qu'on a des cases à visiter
  while cases_a_visiter:

    # S'il faut choisir entre deux cases à visiter
    if len(cases_a_visiter) > 1:
      couts_de_deplacement = []
      # Pour chaque case potentielle à visiter, on calcule le coût.
      for case_potentielle in cases_a_visiter:
        # Le cout actuel est le nombre de déplacement depuis la case départ.
        # Bref la longueur du chemin parcouru entre la case actuel et la case départ.
        chemin = trouverChemin(cases_visitees, cases_precedentes, case_potentielle[1])
        cout_actuel = len(chemin) + 1 # comptant la case potentielle
        cout_estime = heuristique(case_potentielle[0], fin)
        couts_de_deplacement.append(fonctionDeCout(cout_actuel, cout_estime))

      # Choisir le minimum dans la liste de couts de déplacement
      index_cout_min = couts_de_deplacement.index(min(couts_de_deplacement))
      case_courante = cases_a_visiter.pop(index_cout_min)

    # Sinon il n'y a qu'une case à visiter
    else:
      case_courante = cases_a_visiter.pop()

    # On ajoute la case courante aux listes des cases visitées et précédentes
    cases_visitees.insert(0, case_courante[0])
    cases_precedentes.insert(0, case_courante[1])

    # Affiche la case courante dans le graph
    y = int(case_courante[0].center.y)
    x = int(case_courante[0].center.x)
    afficherLabyrinthe(labyrinthe.copy(), x, y, [c.center for c in cases_visitees])

    # Si on trouve la sortie on sort !
    if labyrinthe[y, x] == FIN:
      chemin = trouverChemin(cases_visitees, cases_precedentes, case_courante[0])
      print(f'Nombre de cases visitées: {len(cases_visitees)}')
      print(f'Longueur du chemin: {len(chemin)}')
      return True, chemin

    # Pour chaque case voisine
    cases_voisines = grid.get_node_neighbors(case_courante[0], allow_diagonal=False)
    
    for voisin in cases_voisines:
      if voisin.walkable and voisin not in cases_visitees:
        if voisin not in [c[0] for c in cases_a_visiter]:
          cases_a_visiter.append((voisin, case_courante[0]))
        

  return False, []

# Orde (y, x)
debut = grid.nodes[1][1]
fin = grid.nodes[TAILLE_Y - 2][TAILLE_X - 2]

status, chemin = rechercheHeuristiqueAStar(grid, debut, fin)
afficherChemin(labyrinthe.copy(), chemin)
    